In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# $\mathbb{S}^2$ Sphere Geometry

In [ ]:
from src.manifolds.S2 import *
M = S2()
print(M)
from src.plotting import *
#%matplotlib notebook

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)

# element, tangent vector and covector
x = M.coordsf([0.,0.])
v = tensor([1.,0.])
p = M.flatf(x,v)

print("x = ", x)
print("v = ", v)
print("p = ", p)

## Riemannian Geodesics

In [ ]:
# 2nd order geodesic equation
from src.Riemannian import geodesic
geodesic.initialize(M,do_chart_update=M.do_chart_update)

# compute geodesics
(xs,charts) = M.Exptf(x,v)

# plot
newfig()
M.plot()
M.plot()
M.plot_path(zip(xs,charts),u=v,linewidth = 1.5, s=50)
plt.show()

### Geodesics from Hamiltonian equations

In [ ]:
# Hamiltonian dynamics
q = x
print(M.Hf(q,p))

from src.dynamics import Hamiltonian
Hamiltonian.initialize(M,do_chart_update=M.do_chart_update)

# Exponential map from Hamiltonian equations
(qsT,charts) = M.Exp_Hamiltoniantf(q,p)
qs = qsT.T

# plot
newfig()
M.plot()
M.plot_path(zip(qs,charts),u=v,linewidth = 1.5, s=50)
plt.show()

# dynamics returning both position and momentum
(ts,qps,_) = M.Hamiltonian_dynamicsf(q,p)
ps = qps[:,1,:]
print("Energy: ",np.array([M.Hf((q,chart),p) for (q,p,chart) in zip(qs,ps,charts)]))

## Curvature

In [ ]:
# from src.Riemannian import curvature
# curvature.initialize(M)
# # Curvature tensor, Ricci and scalar curvature:
# print("curvature = ", M.Rf(x))
# print("Ricci curvature = ", M.Ricci_curvf(x))
# print("Scalar curvature = ", M.S_curvf(x))

# # Orthonormal basis under g:
# nu = M.gramSchmidt(x,tensor(np.eye(2))) # or nu = np.linalg.cholesky(M.gsharpf(x))

# # Sectional Curvature
# print("sectional curvature = ",M.sec_curvf(x,nu[:,0],nu[:,1]))

## Parallel Transport

In [ ]:
# Parallel transport
from src.Riemannian import parallel_transport
parallel_transport.initialize(M)

chart = M.chartf()
w = tensor(np.array([-1./2,-1./2]))
w = v/M.normf(x,w)
t = tensor(np.linspace(0,1,n_steps.get_value()))
xs = tensor(np.vstack([t**2,-np.sin(t)]).T)
dxs = tensor(np.vstack([2*t,-np.cos(t)]).T)

# compute  parallel transport
ws = M.parallel_transportf(w,xs,np.tile(chart,(n_steps.eval(),1)),dxs)

# plot result
newfig()
M.plot()
M.plot_path(zip(xs,itertools.cycle((chart,))),v=ws)
plt.show()

# along geodesic
# compute geodesic
(ts,xsdxs,charts) = M.geodesicf(x,v)
xs = xsdxs[:,0,:]
dxs = xsdxs[:,1,:]
# compute  parallel transport
ws = M.parallel_transportf(w,xs,charts,dxs)

# plot
newfig()
M.plot()
M.plot()
M.plot_path(zip(xs,charts),v=ws,linewidth = 1.5, s=50)
plt.show()

## Brownian Motion

In [ ]:
# coordinate form
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M,do_chart_update=M.do_chart_update)

n_steps.set_value(1000)
(ts,xs,charts) = M.Brownian_coordsf(x,dWsf(M.dim.eval()))

# plot
newfig()
M.plot()
M.plot_path(zip(xs,charts))
plt.show()

# plot multiple sample paths
N = 5
xss = tensor(np.zeros((N,n_steps.eval(),M.dim.eval())))
chartss = tensor(np.zeros((N,n_steps.eval(),x[1].shape[0])))
for i in range(N):
    (ts,xs,charts) = M.Brownian_coordsf(x,dWsf(M.dim.eval()))
    xss[i] = xs
    chartss[i] = charts

# plot
M.newfig()
M.plot()
colormap = plt.get_cmap('winter')
colors=[colormap(k) for k in np.linspace(0, 1, N)]
for i in range(N):
    M.plot_path(zip(xss[i],chartss[i]),color=colors[i])
M.plotx(x,color='r',s=50)
plt.show()

In [ ]:
# # Brownian Motion from stochastic development
# # from src.framebundle import FM
# # from src.stochastics import stochastic_development
# # from src.stochastics import Brownian_development

# # FM.initialize(M)
# # stochastic_development.initialize(M)
# # Brownian_development.initialize(M)

# # simulate Brownian Motion
# (ts,xsv) = M.Brownian_developmentf(x,dWsf(M.dim.eval()))

# # plot
# newfig()
# M.plot()
# M.plotx(xsv)
# plt.show()

In [ ]:
# Delyon/Hu guided process
from src.stochastics.guided_process import *

# guide function
Cholesky = T.slinalg.Cholesky()
phi = lambda q,v: T.tensordot(Cholesky(M.gsharp(q)).T,M.StdLog(q,v).flatten(),(1,0))

# plot guiding field
M.newfig()
M.plot_field(lambda x: .2*M.StdLogf(x,tensor([0,0,-1])))
plt.show()

(Brownian_coords_guided,Brownian_coords_guidedf) = get_guided_likelihood(
    M,M.sde_Brownian_coords,phi,lambda x: Cholesky(M.gsharp(x)),
    use_charts=True,chart_update=M.chart_update_Brownian_coords)    

n_steps.set_value(1000)

w = M.Ff(M.Expf(x,tensor(np.array([.8,-.5]))))
(ts,xs,charts,log_likelihood,log_varphi) = Brownian_coords_guidedf(x,w,dWsf(M.dim.eval()))[:5]
print("log likelihood: ", log_likelihood[-1], ", log varphi: ", log_varphi[-1])

# plot
newfig()
M.plot()
M.plot_path(zip(xs,charts))
M.plotx(x,color='r',s=150)
M.plotx(w,color='k',s=150)
plt.show()

# plot multiple bridges
N = 5
w = M.Ff(M.Expf(x,tensor(np.array([.1,-.8]))))
xss = tensor(np.zeros((N,n_steps.eval(),M.dim.eval())))
chartss = tensor(np.zeros((N,n_steps.eval(),x[1].shape[0])))
for i in range(N):
    (ts,xs,charts,log_likelihood,log_varphi) = Brownian_coords_guidedf(x,w,dWsf(M.dim.eval()))[:5]
    xss[i] = xs
    chartss[i] = charts

# plot
M.newfig()
M.plot()
colormap = plt.get_cmap('winter')
colors=[colormap(k) for k in np.linspace(0, 1, N)]
for i in range(N):
    M.plot_path(zip(xss[i],chartss[i]),color=colors[i])
M.plotx(x,color='r',s=100)
M.plotx(w,color='k',s=100)
plt.savefig('S2_bridges.pdf')
plt.show()